In [0]:
# we are going to work on ant vs bee classification
# download data and move it to a `data/` directory

!mkdir data
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip
!mv hymenoptera_data/ data/

In [0]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms, models, datasets

import os
import matplotlib.pyplot as plt
import numpy as np

# Dataset & DataLoader :

train_folder = "data/hymenoptera_data/train"

val_folder = "data/hymenoptera_data/val"

In [0]:
train_data_folder = "data/hymenoptera_data/train"

val_data_folder = "data/hymenoptera_data/val"

transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])])

#create train and val dataset
train_dataset = datasets.ImageFolder(train_data_folder, transform=transform)
val_dataset = datasets.ImageFolder(val_data_folder, transform=transform)

# iterable dataloader
trainloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=1)
valloader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1)


In [0]:
print(f"num of sample in train_dataset : {len(train_dataset)}")
print(f"num of sample in val dataset : {len(val_dataset)}")

# model fine-tune :

In [0]:
# lets use resnet18 model pretrained on imagenet and finetune for our two class classification

model = torchvision.models.resnet18(pretrained=True)
#num of infeatures in fully connected layer
num_features = model.fc.in_features
# change number of classes from 1000 to 2
model.fc = nn.Linear(num_features, 2)

In [0]:
#lets look at the model architechture
model

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.7)

criterion = nn.CrossEntropyLoss()

# training process :

inputs --> forward_pass --> loss_compute --> backward_pass --> update_weights --> repeat

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
num_of_epochs = 10

for epoch in range(num_of_epochs):
  model = model.to(device)
  # set to train mode
  model.train()
  running_loss = 0.0
  correct_preds = 0
  for images, labels in trainloader:
    #access all data
    images, labels = images.to(device), labels.to(device)
    # erase all accumulated gradients
    optimizer.zero_grad()
    # predictions from model
    output = model(images)
    # calculate the loss
    loss = criterion(output, labels)
    #backprop
    loss.backward()
    #update_weights
    optimizer.step()
    # predicion on train dataset
    _, preds = torch.max(output,1)

    running_loss += loss.item()
    correct_preds+=torch.sum(preds == labels.data)

  print(f"finished epoch {epoch}")
  print(f"epoch-{epoch}, loss : {running_loss}, accuracy : {correct_preds.double()/len(train_dataset)}")

In [0]:
classes = train_dataset.classes
print(classes)

In [0]:
# inference on a sinlge validation data in validation dataset.
test_image = val_dataset[-1][0].unsqueeze(0)
test_image_label = val_dataset[-1][1]
# print(test_image.size())
prediction = model(test_image.to(device))
# access the index of predicted class
_, pred = torch.max(prediction,1)
print(classes[test_image_label], classes[pred])

# Evaluate on validation dataset

In [0]:
with torch.no_grad():
  model.eval()
  correct_preds = 0
  for image, labels in valloader:
    image = image.to(device)
    labels = labels.to(device)

    output = model(image)
    _, preds = torch.max(output, 1)
    # print(f"actual : {labels.data}, predicted : {preds}")
    correct_preds+=torch.sum(preds == labels.data)
    # print(f"correct_preds : {correct_preds}")
    val_acc = correct_preds.double() / len(val_dataset)
  print(f"val accuracy : {val_acc}")

# Convnet as fixed feature extractor :

In [0]:
conv_model = models.resnet18(pretrained=True)

for param in conv_model.parameters():
    param.requires_grad = False

num_features = conv_model.fc.in_features

conv_model.fc = nn.Linear(num_features, 2)

conv_model = conv_model.to(device)

criterion = nn.CrossEntropyLoss()

opt_conv = optim.SGD(conv_model.fc.parameters(), lr=0.001, momentum=0.7)

In [0]:
num_of_epochs = 10

for epoch in range(num_of_epochs):
  model = conv_model.to(device)
  # set to train mode
  model.train()
  running_loss = 0.0
  correct_preds = 0
  for images, labels in trainloader:
    #access all data
    images, labels = images.to(device), labels.to(device)
    # erase all accumulated gradients
    optimizer.zero_grad()
    # predictions from model
    output = model(images)
    # calculate the loss
    loss = criterion(output, labels)
    #backprop
    loss.backward()
    #update_weights
    optimizer.step()
    # predicion on train dataset
    _, preds = torch.max(output,1)

    running_loss += loss.item()
    correct_preds+=torch.sum(preds == labels.data)

  print(f"finished epoch {epoch}")
  print(f"epoch-{epoch}, loss : {running_loss}, accuracy : {correct_preds.double()/len(train_dataset)}")

In [0]:
with torch.no_grad():
  model.eval()
  correct_preds = 0
  for image, labels in valloader:
    image = image.to(device)
    labels = labels.to(device)

    output = model(image)
    _, preds = torch.max(output, 1)
    # print(f"actual : {labels.data}, predicted : {preds}")
    correct_preds+=torch.sum(preds == labels.data)
    # print(f"correct_preds : {correct_preds}")
    val_acc = correct_preds.double() / len(val_dataset)
  print(f"val accuracy : {val_acc}")